# Cat Face GAN model  

This is a project to test some of the concepts I have learnt for a Generative Adversial Network (GAN) and also to practise using Git. 

## Step 1: Dataset Inspection 

Since the dataset I've obtained from https://github.com/Ferlix/Cat-faces-dataset should already be standardised to 29,842 64x64 RRB images of cats' faces, there shouldn't be a need to normalise them any further. However, it is still good practice to just take a look at the the pics just to make sure. 

In [1]:
import os 
from matplotlib.image import imread
import numpy as np

In [2]:
%cd "D:/data_science/kaggle/CatFaceGAN"
%pwd

D:\data_science\kaggle\CatFaceGAN


'D:\\data_science\\kaggle\\CatFaceGAN'

In [3]:
imgpath=r'dataset'

In [4]:
dimension1 = []
dimension2 = []
colours = []
for filename in os.listdir(imgpath):
    img = imread(imgpath + '\\' + filename)
    d1,d2,colour = img.shape
    dimension1.append(d1)
    dimension2.append(d2)
    colours.append(colour)

In [5]:
print (np.min(dimension1))
print (np.max(dimension1))
print (np.min(dimension2))
print (np.max(dimension2))
print (np.min(colours))
print (np.max(colours))
print(len(dimension1))

64
64
64
64
3
3
29843


As shown above, it is safe to assume that all 29,843 images are 64x64 and are RGB mode. 

## Step 2: creating GAN model

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Dropout, LeakyReLU

In [8]:
coding_size = 100 
#coding size will be the start of the generator shape 
#it should be smaller than the size of the final image (64x64) but not too small

In [33]:
#generator is a model that takes in noise and creates images 
#these images will try to bypass the classifier (discriminator) and get fake images 
generator = Sequential() 
generator.add(Dense(8*8*128, input_shape=[coding_size]))
generator.add(Reshape([8, 8, 128]))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(64, kernel_size=5, strides=2, 
                              padding="same", activation="relu"))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(64, kernel_size=5, strides=2, 
                              padding="same", activation="relu"))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(3, kernel_size=5, strides=2, 
                              padding="same", activation="relu"))

In [34]:
generator.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 8192)              827392    
_________________________________________________________________
reshape_4 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 8, 8, 128)         512       
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 16, 16, 64)        204864    
_________________________________________________________________
batch_normalization_13 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 32, 32, 64)        102464    
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 64)       

In [35]:
#discriminator is a classification model, employing typical CNN techniques 
discriminator = Sequential() 
discriminator.add(Conv2D(64, kernel_size=5, strides=1, padding="same",
                        activation=LeakyReLU(0.3),
                        input_shape=[64, 64, 3]))
discriminator.add(MaxPooling2D(2,2))
discriminator.add(Dropout(0.5))
discriminator.add(Conv2D(64, kernel_size=5, strides=1, padding="same",
                        activation=LeakyReLU(0.3)))
discriminator.add(MaxPooling2D(2,2))
discriminator.add(Dropout(0.5))
discriminator.add(Conv2D(64, kernel_size=5, strides=2, padding="same",
                        activation=LeakyReLU(0.3)))
discriminator.add(Dropout(0.5))
discriminator.add(Flatten())
discriminator.add(Dense(64, activation='relu'))
discriminator.add(Dense(64, activation='relu'))
discriminator.add(Dense(1, activation="sigmoid"))

In [36]:
discriminator.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        102464    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 64)         

In [37]:
GAN=Sequential([generator,discriminator])

In [38]:
discriminator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.trainable = False

In [39]:
GAN.compile(loss="binary_crossentropy", optimizer="adam")

## Step 3: loading the images into train data 

In [40]:
training_data = []

for filename in os.listdir(imgpath):
    img = imread(imgpath + '\\' + filename)
    training_data.append(img)

In [41]:
X_train = np.array(training_data).reshape(-1,64,64,3)

In [42]:
X_train.shape

(29843, 64, 64, 3)

In [43]:
scaled_train = X_train/255

## Step 4: training GAN model 

In [44]:
batch_size = 32
my_data = scaled_train

dataset= tf.data.Dataset.from_tensor_slices(my_data)
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size,drop_remainder=True)
epochs = 20

generator, discriminator = GAN.layers
for epoch in range(epochs):
      print(f"Currently on Epoch {epoch+1}")
      i = 0
      for X_batch in dataset:
            i = i+1
            if i%100 ==0:
                print(f"\t Currently on batch number {i} of {len(my_data)//batch_size}")
      # Discriminator 
      noise = tf.random.normal(shape=[batch_size,coding_size])
      gen_images = generator(noise)
      X_fake_vs_real = tf.concat([gen_images, tf.dtypes.cast(X_batch, tf.float32)], axis=0)
      y1 = tf.constant([[0.0]*batch_size]+ [[1.0]* batch_size]) #0 for the fake images and 1 for real]]
      discriminator.trainable = True
      discriminator.train_on_batch(X_fake_vs_real, y1)

      # Generator
      noise = tf.random.normal(shape=[batch_size, coding_size])
      y2 = tf.constant([[1.0]*batch_size])
      discriminator.trainable = False 
      GAN.train_on_batch(noise,y2)

Currently on Epoch 1
	 Currently on batch number 100 of 932
	 Currently on batch number 200 of 932
	 Currently on batch number 300 of 932
	 Currently on batch number 400 of 932
	 Currently on batch number 500 of 932
	 Currently on batch number 600 of 932
	 Currently on batch number 700 of 932
	 Currently on batch number 800 of 932
	 Currently on batch number 900 of 932


ValueError: in converted code:

    C:\Users\antho\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training_eager.py:305 train_on_batch  *
        outs, total_loss, output_losses, masks = (
    C:\Users\antho\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training_eager.py:253 _process_single_batch
        training=training))
    C:\Users\antho\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training_eager.py:167 _model_loss
        per_sample_losses = loss_fn.call(targets[i], outs[i])
    C:\Users\antho\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\losses.py:221 call
        return self.fn(y_true, y_pred, **self._fn_kwargs)
    C:\Users\antho\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\losses.py:994 binary_crossentropy
        K.binary_crossentropy(y_true, y_pred, from_logits=from_logits), axis=-1)
    C:\Users\antho\Anaconda3\envs\te